In [ ]:
#寫入野球革命2023年逐場打擊數據

import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 初始化 Edge 驅動程序
driver = webdriver.Edge()

# 打開網頁
driver.get("https://www.rebas.tw/player/0fd4d?tab=Log&type=batter")

# 使用显式等待等待動態內容加載完成，此處等待了10秒，根據實際情況進行調整
wait = WebDriverWait(driver, 10)

# 獲取所有 <tbody> 元素
tbody_elements = driver.find_elements(By.ID, "formal-player-log-tab")

# 準備要寫入 CSV 檔案的資料
csv_data = []

# 輸出 <tbody> 元素的內容
for tbody in tbody_elements:
    # 將每一行的文本分割成列
    rows = tbody.text.split('\n')
    for idx, row in enumerate(rows):
        # 跳過第二筆資料
        if idx == 0:
            continue
        
        # 將每一列的文本分割成單元格
        cells = row.split()  # 根據實際情況修改分隔符號
        
        # 加入到 CSV 資料中
        csv_data.append(cells)


# 關閉瀏覽器窗口
driver.quit()

# 將資料寫入 CSV 檔案
csv_file_path = 'ya_2023.csv'
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # 寫入表頭
    csv_writer.writerow(['日期', '場次', '對手', '打席', '打數', '得分', '安打', '全壘打', '打點', '四壞', '故四', '觸身' ,'三振' ,'犧觸', '犧飛', '雙殺' , 'RE24', 'WPA']) 
    
    # 輸出資料行
    for row in csv_data:
        csv_writer.writerow(row)


In [45]:
#寫入野球革命2022年逐場打擊數據

import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

# 初始化 Edge 驅動程序
driver = webdriver.Edge()

# 打開網頁
driver.get("https://www.rebas.tw/player/0fd4d?tab=Log&type=batter")

# 使用显式等待等待動態內容加載完成，此處等待了10秒，根據實際情況進行調整
wait = WebDriverWait(driver, 10)

# 等待下拉菜单加载完成
dropdown = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "global-option--season")))

# 使用 Select 类来操作下拉菜单
select = Select(dropdown)

# 选择年份（假设年份是2022）
select.select_by_value("CPBL-2022-dG")

# 再次等待动态内容加载完成，根据实际情况进行调整
wait.until(EC.presence_of_all_elements_located((By.ID, "formal-player-log-tab")))

# 獲取所有 <tbody> 元素
tbody_elements = driver.find_elements(By.ID, "formal-player-log-tab")

# 準備要寫入 CSV 檔案的資料
csv_data = []

# 輸出 <tbody> 元素的內容
for tbody in tbody_elements:
    # 將每一行的文本分割成列
    rows = tbody.text.split('\n')
    for idx, row in enumerate(rows):
        # 跳過第二筆資料
        if idx == 0:
            continue
        
        # 將每一列的文本分割成單元格
        cells = row.split()  # 根據實際情況修改分隔符號
        
        # 加入到 CSV 資料中
        csv_data.append(cells)


# 關閉瀏覽器窗口
driver.quit()

# 將資料寫入 CSV 檔案，附加到末尾
csv_file_path = 'ya.csv'
with open(csv_file_path, 'a', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # 輸出資料行
    for row in csv_data:
        csv_writer.writerow(row)

The msedgedriver version (119.0.2151.97) detected in PATH at c:\Rachel\GitHub\CPBL_Baseball_ML_project\msedgedriver.exe might not be compatible with the detected MicrosoftEdge version (120.0.2210.61); currently, msedgedriver 120.0.2210.61 is recommended for MicrosoftEdge 120.*, so it is advised to delete the driver in PATH and retry


In [49]:
#因為資料來源是包含當年度所有編號的例行賽
#當選手當天沒有上場打擊時，資料內容會出現「-」
#刪除內容包含「-」的資料
#計算並加入打擊率欄位

import csv

# 舊的 CSV 檔案路徑
old_csv_file_path = 'cpbl_05.csv'

# 讀取舊的 CSV 檔案
with open(old_csv_file_path, 'r', newline='', encoding='utf-8') as old_csvfile:
    # 使用 csv.reader 讀取舊檔案的內容
    old_csv_data = list(csv.reader(old_csvfile))

# 準備新的 CSV 檔案的資料
new_csv_data = []

# 寫入表頭
new_csv_data.append(['日期', '場次', '對手', '打席', '打數', '得分', '安打', '全壘打', '打點', '四壞', '故四', '觸身' ,'三振' ,'犧觸', '犧飛', '雙殺' , 'RE24', 'WPA', '打擊率'])

# 處理舊檔案的每一列資料
for old_row in old_csv_data[1:]:  # 跳過表頭
    # 如果打席欄位包含 "-"
    if '-' in old_row[3]:  # 假設打席欄位在第4個位置
        # 刪除包含 "-" 的整個資料行
        continue
    
    # 將每一列的文本分割成單元格
    cells = old_row
    
    # 這裡可以加入計算打擊率的邏輯，假設打擊率的計算公式為安打數/打數
    hits = int(cells[6])  # 假設安打數在第7個位置
    at_bats = int(cells[5])  # 假設打數在第6個位置
    
    batting_average = hits / at_bats if at_bats > 0 else 0  # 計算打擊率，避免除以零
    
    # 加入到新 CSV 資料中
    new_csv_data.append(cells + [f'{batting_average:.3f}'])  # 將打擊率加入到新的資料行中

# 新的 CSV 檔案路徑
new_csv_file_path = 'new_cpbl_05.csv'

# 寫入新的 CSV 檔案
with open(new_csv_file_path, 'w', newline='', encoding='utf-8') as new_csvfile:
    csv_writer = csv.writer(new_csvfile)
    
    # 輸出所有新資料行
    for new_row in new_csv_data:
        csv_writer.writerow(new_row)



In [1]:
#刪除異常欄位(有4-5筆資料多出一個無意義的欄位資料)

import csv

def remove_column_20(input_file, output_file):
    with open(input_file, 'r', newline='', encoding='utf-8') as infile:
        with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
            reader = csv.reader(infile)
            writer = csv.writer(outfile)

            for row in reader:
                # 如果行中有 20 個欄位，就刪除第 20 個欄位
                if len(row) == 20:
                    del row[19]

                # 寫入處理後的行
                writer.writerow(row)

# 指定輸入檔案和輸出檔案
input_file_path = 'new_cpbl_05.csv'
output_file_path = 'cpbl_rakuten_05.csv'

# 執行刪除第 20 欄位的操作
remove_column_20(input_file_path, output_file_path)

In [30]:
#查看檔案資料型別
import pandas as pd

# 讀取 CSV 檔案成 DataFrame
csv_file_path = 'ya_2023.csv'
df = pd.read_csv(csv_file_path)

# 顯示 DataFrame 的資料型別
print(df.dtypes)


日期       object
場次        int64
對手       object
打席        int64
打數        int64
得分        int64
安打        int64
全壘打       int64
打點        int64
四壞        int64
故四        int64
觸身        int64
三振        int64
犧觸        int64
犧飛        int64
雙殺        int64
RE24    float64
WPA     float64
dtype: object
